In [14]:
import spacy
from spacy import displacy
from spacy.pipeline import DependencyParser
from collections import Counter
# import en_core_web_sm
from pprint import pprint
nlp = spacy.load('en_core_web_sm')

c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from P

In [15]:
# sampleText = u"McCoy has now been held below 10 touches in 2-of-3 games and has handled the ball only 29 times all season."
# sampleText = "Mark Ingram returned from his four-week suspension to make his 2018 debut and made the start for the Saints."
sampleText = "Kamara got the start and out-touched Mark Ingram 19-14 one game after Ingram out-touched Kamara 18-9 in his 2018 debut. Both players received goal line looks, but only Kamara cashed his in."

In [16]:
# Another sample
doc = nlp('European authorities fined Google a record $5.1 billion on' +
          'Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
pprint([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'),
 ('Google', 'ORG'),
 ('$5.1 billion', 'MONEY'),
 ('onWednesday', 'DATE')]


In [28]:
doc = nlp(sampleText)
pprint([(X.text, X.label_) for X in doc.ents])

[('Tavon Austin', 'PERSON'),
 ('two', 'CARDINAL'),
 ('79 yards', 'QUANTITY'),
 ('Week 2', 'DATE'),
 ('Giants', 'ORG')]


In [18]:
# pprint([(X, X.ent_iob_, X.ent_type_) for X in doc])

In [29]:
# NER
displacy.render(nlp(sampleText), jupyter=True, style='ent')
dict([(str(x), x.label_) for x in nlp(sampleText).ents])

{'79 yards': 'QUANTITY',
 'Giants': 'ORG',
 'Tavon Austin': 'PERSON',
 'Week 2': 'DATE',
 'two': 'CARDINAL'}

In [30]:
for token in doc:
    print(token.text, token.i, token.pos_, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children])

Tavon 0 PROPN compound Austin PROPN []
Austin 1 PROPN nsubj caught VERB [Tavon]
caught 2 VERB ROOT caught VERB [Austin, passes, against, .]
two 3 NUM nummod passes VERB []
passes 4 VERB dobj caught VERB [two, for]
for 5 ADP prep passes VERB [yards]
79 6 NUM nummod yards NOUN []
yards 7 NOUN pobj for ADP [79, and, touchdown]
and 8 CCONJ cc yards NOUN []
a 9 DET det touchdown NOUN []
touchdown 10 NOUN conj yards NOUN [a, in]
in 11 ADP prep touchdown NOUN [Week]
Week 12 PROPN pobj in ADP [2]
2 13 NUM nummod Week PROPN []
against 14 ADP prep caught VERB [Giants]
the 15 DET det Giants PROPN []
Giants 16 PROPN pobj against ADP [the]
. 17 PUNCT punct caught VERB []


In [21]:
# If CCONJ has cc dep on PROPN's nsubjpass, break sentence as:
subSentences = {}
subSentenceCounter = 0
subSentences[subSentenceCounter] = []
subject = None
propn = None

for token in doc:
    if token.pos_ == 'PROPN' and (token.dep_ == 'nsubjpass' or token.dep_ == 'nsubj'):
        subject = token.head.text
        propn = token.text
        
    if token.pos_ == 'CCONJ' and token.dep_ == 'cc' and token.head.text == subject:
        subSentences[subSentenceCounter].append('.')
        subSentenceCounter += 1
        subSentences[subSentenceCounter] = []
        subSentences[subSentenceCounter].append(propn)
    else:
        subSentences[subSentenceCounter].append(token.text)
        
for k,v in subSentences.items():
    print(" ".join(v).replace(' .', '.').replace(' - ', '-').replace(' ,', ','))

Kamara got the start and out-touched Mark Ingram 19-14 one game after Ingram out-touched Kamara 18-9 in his 2018 debut. Both players received goal line looks, but only Kamara cashed his in.


In [22]:
# Dependencies graph
displacy.render(nlp(sampleText), style='dep', jupyter = True, options = {'distance': 120})

In [23]:
# POS and lemmatizing
[(x.orth_,x.pos_, x.lemma_) for x in [y for y in nlp(sampleText) if not y.is_stop and y.pos_ != 'PUNCT']]

[('Kamara', 'PROPN', 'kamara'),
 ('got', 'VERB', 'get'),
 ('start', 'NOUN', 'start'),
 ('touched', 'VERB', 'touch'),
 ('Mark', 'PROPN', 'mark'),
 ('Ingram', 'PROPN', 'ingram'),
 ('19', 'NUM', '19'),
 ('-', 'SYM', '-'),
 ('14', 'NUM', '14'),
 ('game', 'NOUN', 'game'),
 ('Ingram', 'PROPN', 'ingram'),
 ('touched', 'VERB', 'touch'),
 ('Kamara', 'PROPN', 'kamara'),
 ('18', 'NUM', '18'),
 ('-', 'SYM', '-'),
 ('9', 'NUM', '9'),
 ('2018', 'NUM', '2018'),
 ('debut', 'NOUN', 'debut'),
 ('Both', 'DET', 'both'),
 ('players', 'NOUN', 'player'),
 ('received', 'VERB', 'receive'),
 ('goal', 'NOUN', 'goal'),
 ('line', 'NOUN', 'line'),
 ('looks', 'VERB', 'look'),
 ('Kamara', 'PROPN', 'kamara'),
 ('cashed', 'VERB', 'cash')]

In [48]:
import csv
import numpy as np

In [86]:
# Load file to be chunked
reports = []

with open('../../../analysis/data/output_templates.csv', encoding='utf-8-sig') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            reports.append(row[1])
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are reports, templates
Processed 1240 lines.


In [26]:
sampleText = reports[0]

In [88]:
# Chunk sentences by prepositions
def chunkText(text):
    retval = []
    subSentences = {}
    subSentenceCounter = 0
    subSentences[subSentenceCounter] = []
    textDoc = nlp(text)
    
    for token in textDoc:
        if token.dep_ == 'prep':
    #         subSentences[subSentenceCounter].append('.')
            subSentenceCounter += 1
            subSentences[subSentenceCounter] = []
        subSentences[subSentenceCounter].append(token.text)

    for k,v in subSentences.items():
        retval.append(" ".join(v).replace(' .', '').replace(" '", "'").replace(" ’", "'").replace('$ { ', '${').replace(' }', '}'))
    
    return retval

In [90]:
# Write chunks to file
with open('../../../analysis/data/output_chunked.csv', mode='w') as chunk_file:
    chunk_writer = csv.writer(chunk_file, delimiter=',', lineterminator='\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for text in reports:
        chunks = chunkText(text)
        chunk_writer.writerow(chunks)


In [89]:
for text in reports[:2]:
    chunks = chunkText(text)
    print(chunks)

['${player_name} caught ${temp_var_0} passes', 'for ${rec_yards} yards and a touchdown', 'in week ${temp_var_1}', 'against the ${opp}.']
['${player_name} delivered ${temp_var_0} catches', 'for ${rec_yards} yards and a touchdown while gaining ${temp_var_1} yards', 'on his lone carry Sunday', "in the ${team}' week ${temp_var_2} loss", 'to the ${opp}.']
